# Class - Convolutional Neural Networks 2 

In this class, you will load MobileNetV2: use it to predict the class of the image of your choice. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
from PIL import Image
from io import BytesIO
#
import tensorflow as tf
from sklearn.model_selection import train_test_split  
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import  preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions
#
import tensorflow_datasets as tfds

In [ ]:
print(tf.__version__)
print(tf.keras.__version__)

You will create the base model from the MobileNet V2 model developed at Google. This is pre-trained on the ImageNet dataset, a large dataset consisting of 1.4M images and 1000 classes. ImageNet is a research training dataset with a wide variety of categories like jackfruit and book store. 

In [ ]:
def load_image_from_url(url):
    """Returns an image with shape [height, width, num_channels]."""
    response = requests.get(url)
    image    = Image.open(BytesIO(response.content))
    image    = np.array(image)
    return image

def show_image(image,ax=None):
    if ax==None: ax=plt.gca()
    ax.imshow(image);
    ax.axis('off')

## Choose 4 images located on the web to predict the class label 

In [ ]:
# Load image 
img_url1 = 'http://public.gettysburg.edu/~jpuckett/ds325/img/laney.jpg'
img_url2 = 'http://public.gettysburg.edu/~jpuckett/ds325/img/togo.jpg'
img_url3 = 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Cherry_blossom_flowers_1.jpg/724px-Cherry_blossom_flowers_1.jpg'
img_url4 = 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/Ring-tailed_lemur_%28Lemur_catta%29.jpg/683px-Ring-tailed_lemur_%28Lemur_catta%29.jpg'

#
img1     = load_image_from_url(img_url1)
img2     = load_image_from_url(img_url2)
img3     = load_image_from_url(img_url3)
img4     = load_image_from_url(img_url4)
show_image(img1)

#### Let's take a look at the base model architecture


In [ ]:
base_model =tf.keras.applications.MobileNetV2( include_top=True, weights='imagenet',input_shape=(224,224,3))
base_model.trainable = False
base_model.summary()

* The model expects the image to be in the batch format (#batch,img_width,img_height,#channels)
* The input need to be pre-processed

In [ ]:
# process an image to be mobilenet friendly
def process_image(img,IMG_SIZE):
    imgR              = tf.image.resize(img,(IMG_SIZE,IMG_SIZE))    
    img_array         = np.array(imgR)
    img_array         = np.expand_dims(img_array, axis=0)
    pImg              = preprocess_input(img_array)
    return pImg
def classify_images(base_model,img,IMG_SIZE=224):
    #preprocess image
    imgA              = process_image(img,IMG_SIZE)
    prediction        = base_model.predict(imgA)
    #output from model to determin class
    top3              = decode_predictions(prediction, top=3)[0]
    return top3

In [ ]:
top3 = classify_images(base_model,img1)
print(top3)

In [ ]:
#put it all together to show the image and get the label
def imshowCaption(base_model,img,ax=None):
    top3 = classify_images(base_model,img)
    if ax is None: ax=plt.gca()
    show_image(img)
    #make top choice str
    lbl = top3[0][1]
    acc = top3[0][2]    
    str1 = '%s with accuracy=%2.1f%%'%(lbl,acc*100)
    #make 2nd choice str
    lbl = top3[1][1]
    acc = top3[1][2]    
    str2 = '%s with accuracy=%2.1f%%'%(lbl,acc*100)
    lbl = top3[2][1]
    acc = top3[2][2]    
    str3 = '%s with accuracy=%2.1f%%'%(lbl,acc*100)
    
    plt.title('%s\n%s\n%s'%(str1,str2,str3))

In [ ]:
imshowCaption(base_model,img1)

In [ ]:
plt.figure(figsize=(12, 8))
ax1=plt.subplot(221)
imshowCaption(base_model,img1,ax1)
ax2=plt.subplot(222)
imshowCaption(base_model,img2,ax2)
ax3=plt.subplot(223)
imshowCaption(base_model,img3,ax3)
ax4=plt.subplot(224)
imshowCaption(base_model,img4,ax4)
plt.tight_layout()

MobileNetV2 seems to be classifying the images with great accuracy. 


* Togo is mislabeled, should be 'Eskimo dog, husky'.  'kelpie' is an 'Australian kelpie', not the mythological creature.
* The 'madagascar_cat' is otherwise known as a ring-taled lemur.
* It was not very good at identifying the cherry blossoms.  There are surprisingly few flower species in the imagenet dataset.  We'll get to that below.

While MobileNetV2 did a great job with these, one of your hw problems is for you to test a few images yourself.

# Transfer learning 

Transfer learning involves taking a pre-trained model, extracting one of the layers, then taking that as the input layer to a series of dense layers. This pre-trained model is usually trained by institutions or companies that have much larger computation and financial resources. Some of these popular trained models for image recognition tasks are VGG, Inception, ResNet and MobileNet.

Using this newly formed model, we can then set the parameters within the pre-trained model to be non-trainable while only optimizing the parameters of the subsequent dense layers during training.

In [ ]:
import tensorflow_datasets as tfds
(test_set_raw, valid_set_raw, train_set_raw),info = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True,with_info=True)
class_names   = info.features["label"].names
n_classes     = info.features["label"].num_classes
dataset_size  = info.splits["train"].num_examples

Basic preprocessing:

In [ ]:
def preprocess(image, label,IMG_SHAPE=224):
    resized_image = tf.image.resize(image, [IMG_SHAPE, IMG_SHAPE])
    final_image = tf.keras.applications.mobilenet_v2.preprocess_input(resized_image)
    return final_image, label

In [ ]:
batch_size = 32
train_set = train_set_raw.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set  = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [ ]:
plt.figure(figsize=(12, 12))
for X_batch, y_batch in train_set.take(1):
    for index in range(9):
        plt.subplot(3, 3, index + 1)
        plt.imshow(X_batch[index] / 2 + 0.5)
        plt.title("Class: {}".format(class_names[y_batch[index]]))
        plt.axis("off")

plt.show()

## Build our model

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE  = 224
base_model2 = tf.keras.applications.MobileNetV2(input_shape=(IMG_SHAPE,IMG_SHAPE,3),
                                              include_top=False, 
                                              weights='imagenet')
for layer in base_model2.layers:
    layer.trainable = False
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model2.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
#now construct our model from these layers, we could add a hidden layer, but lets try something simple first
model = tf.keras.models.Model(inputs=base_model2.input, outputs=output)

In [ ]:
for index, layer in enumerate(base_model2.layers):
    print(index, layer.name)

## Compile the model

In [ ]:
optimizer = tf.keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

## Fit the model.  
#### Note: on my laptop, this takes about 10s/epoch
* We are only using 1/10th the data for each epoch.  Since we are more interested in the concept than the optimally trained model.
* This gives us better visualization of how the model is learning.  

In [ ]:
epochs   = 6
tstart   = tf.timestamp()
history  = model.fit(train_set,
                    steps_per_epoch =int(0.1 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.05 * dataset_size / batch_size),
                    epochs=epochs)
total_time = tf.timestamp() - tstart
print("total time %3.3f seconds"%total_time)

In [ ]:
def printAccuracy(history,results_test):
    print("train loss %.5f \t train acc: %.5f"%(history.history['loss'][-1],history.history['accuracy'][-1]))
    print("valid loss %.5f \t valid acc: %.5f"%(history.history['val_loss'][-1],history.history['val_accuracy'][-1]))
    print("test loss  %.5f \t test acc:  %.5f"%(results_test[0],results_test[1]))
def plot_result(history,results_test):
    # Get training and validation histories
    training_acc = history.history['accuracy']
    val_acc      = history.history['val_accuracy']
    # Create count of the number of epochs
    epoch_count = range(1, len(training_acc) + 1)
    # Visualize loss history
    plt.plot(epoch_count, training_acc, 'b-o',label='Training')
    plt.plot(epoch_count, val_acc, 'r--',label='Validation')
    plt.plot(epoch_count, results_test[1]*np.ones(len(epoch_count)),'k--',label='Test')
    plt.legend()
    plt.title("Training and validation accuracy")
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
 

In [ ]:
#===    
results_test = model.evaluate(test_set, batch_size=128,verbose=0)    
printAccuracy(history,results_test)

In [ ]:
plot_result(history,results_test)   
plt.title("Transfer learning using MobileNetV2, acc=%2.3f, in %3.2f s"%(results_test[1],total_time)) #overwrite the title
plt.show()

Doesn't look like we are overtraining.  85% is remarkable given <2min of training.